This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'UKQAVipGFcy3aw1x47dd'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [126]:
# First, import the relevant modules
import requests


In [9]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/' + 'FSE/AFX_X/' +\
                 '/data.json?&start_date=2018-12-21&end_date=2018-12-21' +\
                 '&api_key=' + API_KEY)

In [10]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'collapse': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'data': [['2018-12-21',
    None,
    69.3,
    67.9,
    68.95,
    None,
    None,
    None,
    None,
    None,
    None]],
  'end_date': '2018-12-21',
  'frequency': 'daily',
  'limit': None,
  'order': None,
  'start_date': '2018-12-21',
  'transform': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [206]:
# 1. 
# Make API request return json data
r = requests.get('https://www.quandl.com/api/v3/datasets/' + 'FSE/AFX_X/' +\
                 '/data.json?&start_date=2017-01-01&end_date=2017-12-31' +\
                 '&api_key=' + API_KEY)

json_data = r.json()

In [207]:
# 2.
#cast json_data as a dictionary
dict_data = dict(json_data['dataset_data'])

In [208]:
#create function that creates a list of a certain column from the json data
def column_list(column_index):
    '''
    Use column index below to create a list of that attribute for 
    Date:0,Open:1,High:2,Low:3,Close:4,Traded Volume:6,Turnover:7
    
    dict_data['data'][index_of_certain_day][attribute_index]
    
    column_index: int
    returns: list
    '''
    return [dict_data['data'][x][column_index] for x in range(len(dict_data['data']))]

In [210]:
# 3.
#Print the highest and lowest open price foro AFX_X in 2017
#Data for Open contains None values, filter out None values to be able to call min() and max()

highest_open_price = max(x for x in column_list(1) if x is not None)
lowest_open_price = min(x for x in column_list(1) if x is not None)

print('The highest open price for AFX_X in 2017 was ' + str(highest_open_price))
print('The lowest open price for AFX_X in 2017 was ' + str(lowest_open_price))

The highest open price for AFX_X in 2017 was 53.11
The lowest open price for AFX_X in 2017 was 34.0


In [211]:
# 4.
#Using list comprehension to make a new list of (highs - lows) for each day
change = [column_list(2)[x] - column_list(3)[x] for x in range(len(dict_data['data']))]

print('The largest price fluctuation in any one day in 2017 was '+ str(max(change)))

The largest price fluctuation in any one day in 2017 was 2.8100000000000023


In [212]:
# 5.
#To find the max and min closing prices in 2017 and take the difference to find largest change
closing_prices = column_list(4)
max_close = max(closing_prices)
min_close = min(closing_prices)
largest_change_2days = max_close - min_close

print('The largest change based on close price between any two days in 2017 was ' + str(largest_change_2days))

The largest change based on close price between any two days in 2017 was 19.03


In [213]:
# 6.
#Find average trade volume by calculting sum of total trade volume divided by the 
trade_volumes_per_day = column_list(6)
average_trade_vol = sum(trade_volumes_per_day) / len(trade_volumes_per_day)
print('The average trade volume for 2017 was ' + str(average_trade_vol))



The average trade volume for 2017 was 89124.33725490196


In [214]:
# Define function to calculate the median

def median(col_list):
    '''
    Returns the median from a list of numerical values
    '''
    #sort the list without modifying the orginal list
    sorted_list = sorted(col_list)
    if len(sorted_list) % 2 == 0:
        return (sorted_list[int(len(sorted_list)/2)] +\
                sorted_list[int(len(sorted_list)/2 - 1)]) / 2
    else:
        return sorted_list[len(sorted_list)//2]

In [215]:
# 7.

#sort the list of daily trade volumes without modifying the original list, then 
median_trade_vol = median(sorted_avg_trade_vol)
print('The median trade volume for 2017 was ' + str(median_trade_vol))

The median trade volume for 2017 was 76286.0
